In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from math import *
import numpy.random as npr

##################################################
######## Estimation de Pi par Monte Carlo ########
##################################################
def tirage(m):
   Naleat = np.random.uniform (-m, m, 2)
   x=Naleat[0]
   y=Naleat[1]
   return x,y

def monteCarlo(N):
    m = 1.0
    s=0
    coordx = []
    coordy = []
    for i in range(N):
        x,y = tirage(m)
        coordx.append(x)
        coordy.append(y)
        if (x**2 + y**2) <= 1:
            s += 1.0
    Pi = (s/N)*4
    return Pi, np.array(coordx),np.array(coordy)


def affichMonteCarlo():
    plt.figure()
    # trace le carré
    plt.plot([-1, -1, 1, 1], [-1, 1, 1, -1], '-')
    # trace le cercle
    x = np.linspace(-1, 1, 100)
    y = np.sqrt(1- x*x)
    plt.plot(x, y, 'b')
    plt.plot(x, -y, 'b')
    # estimation par Monte Carlo
    pi, x, y = monteCarlo(int(1e4))
    # trace les points dans le cercle et hors du cercle
    dist = x*x + y*y
    plt.plot(x[dist <=1], y[dist <=1], "go")
    plt.plot(x[dist>1], y[dist>1], "ro")
    plt.show()
        
#affichMonteCarlo()
##################################################
###############Decodage par Metropolis-Hasting ###############
##################################################

#Questions 1, 2 : Telechargement et execution du modele de bigrammes et du texte
with open("countWar.pkl", 'rb') as f:
    (count, mu, A) = pkl.load(f, encoding='latin1')

with open("secret.txt", 'r') as f:
    secret = f.read()[0:-1] # -1 pour supprimer le saut de ligne
with open("secret2.txt", 'r') as f:
    secret2 = f.read()[0:-1]
#Question 3: fonction swapF 
def swapF(tau):
    tau1 = {}
    n1 = np.random.randint(0,len(tau))
    n2 = np.random.randint(0,len(tau))
    while(n2==n1):
        n2 = np.random.randint(0,len(tau))
    k1 = list(tau)[n1]
    k2 = list(tau)[n2]
    tau1[k1] = tau[k2]
    tau1[k2] = tau[k1]
    for i in tau.keys():
        if i != k1 and i != k2:
            tau1[i] = tau[i]
    return tau1

#tau = {'a' : 'b', 'b' : 'c', 'c' : 'a', 'd' : 'd' }
#swapF (tau)

#Question 4: fonction decrypt
def decrypt(mess,tau):
    mess1 = ""
    for i in mess:
        mess1 += tau[i]
    
    return mess1

#print(decrypt ( "aabcd", tau ))
#print(decrypt( "dcba", tau ))

#Question 5: création d'une table de hashage

chars2index = dict(zip(np.array(list(count.keys())), np.arange(len(count.keys()))))

#Question 6: fonction logLikelihood
def logLikelihood(mess,mu,A,chars):
	i = chars.index(mess[0])
	logv = np.log(mu[i])
	for c in range(1,len(mess)):
		j = chars.index(mess[c])
		logv+= np.log(A[i][j])
		i = j
	return logv
print(logLikelihood( "abcd", mu, A, list(chars2index)))
#print(logLikelihood( "dcba", mu, A, list(count.keys())))


#Question 7: codage de la fonction Metropolis Hastings
#meilleurs résultats..
def MetropolisHastings(mess, mu, A, tau, N):
    messdecode = decrypt(mess,tau)
    logVbest = logLikelihood(messdecode, mu, A, list(count.keys ()))
    for i in range(N):
        tau1 = swapF(tau)
        messdecode = decrypt(mess,tau1)
        logv = logLikelihood(messdecode, mu, A, list(count.keys ()))
        
        if logv > logVbest:
            #On garde modification de tau
            logVbest = logv
            tau = tau1
    
    messdecode = decrypt(mess,tau)
    return messdecode


def MetropolisHastings2(mess, mu, A, tau, N):
    messdecode = decrypt(mess,tau)
    logV = logLikelihood(messdecode, mu, A, list(count.keys ()))
    for i in range(N):
        tauNew = swapF(tau)
        messdecode = decrypt(mess,tauNew)
        logVNew = logLikelihood(messdecode, mu, A, list(count.keys ()))
        #meilleurs résultats sans le exp...
        alpha=min(1,(logVNew/logV) )
        p = np.random.uniform()
        
        if p < alpha:
            logV = logVNew
            tau = tauNew
    
    messdecode = decrypt(mess,tau)
    return messdecode
    
    
def identityTau ():
    tau = {}
    for k in count.keys ():
        tau[k] = k
    return tau
    
#print MetropolisHastings( secret2, mu, A, identityTau(), 10000 )

#Q7
def updateOccurrences(text, count):
   for c in text:
      if c == u'\n':
         continue
      try:
         count[c] += 1
      except KeyError as e:
         count[c] = 1

def mostFrequent(count):
   bestK = []
   bestN = -1
   for k in count.keys():
      if (count[k]>bestN):
         bestK = [k]
         bestN = count[k]
      elif (count[k]==bestN):
         bestK.append(k)
   return bestK

def replaceF(f, kM, k):
   try:
      for c in f.keys():
         if f[c] == k:
            f[c] = f[kM]
            f[kM] = k
            return
   except KeyError as e:
      f[kM] = k

def mostFrequentF(message, count1, f={}):
   count = dict(count1)
   countM = {}
   updateOccurrences(message, countM)
   while len(countM) > 0:
      bestKM = mostFrequent(countM)
      bestK = mostFrequent(count)
      if len(bestKM)==1:
         kM = bestKM[0]
      else:
         #kM = bestKM[npr.random_integers(0, len(bestKM)-1)]
         kM = bestKM[npr.randint(0, len(bestKM)-1)]
      if len(bestK)==1:
         k = bestK[0]
      else:
         #k = bestK[npr.random_integers(0, len(bestK)-1)]
         k = bestK[npr.randint(0, len(bestK)-1)]
      replaceF(f, kM, k)
      countM.pop(kM)
      count.pop(k)
   return f
